# Maintenance Report NLP Analysis

This notebook implements an NLP pipeline for analyzing maintenance reports using BERT and NLTK.

In [1]:
import os
import platform
import re
import torch
import pandas as pd
from transformers import BertTokenizer, BertForTokenClassification, pipeline, logging
from spellchecker import SpellChecker
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Suppress warnings from the transformers library
logging.set_verbosity_error()

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet', quiet=True)

try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger', quiet=True)

In [2]:
# Check if GPU is available
if platform.system() == 'Darwin':  # macOS
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = torch.device("mps")
        print("MPS backend is available. PyTorch is using the GPU.")
    else:
        device = torch.device("cpu")
        print("MPS backend is not available. PyTorch is using the CPU.")
else:  # Windows or other platforms
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("CUDA backend is available. PyTorch is using the GPU.")
    else:
        device = torch.device("cpu")
        print("CUDA backend is not available. PyTorch is using the CPU.")

MPS backend is available. PyTorch is using the GPU.


In [3]:
import pyodbc
import pandas as pd

In [4]:
# Step: Load Data

# Define the path to ice makers' data file
csv_file_path = 'data/ice_makers.csv'

# Check if the data file exists
if os.path.exists(csv_file_path):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)
    print(f"DataFrame loaded from {csv_file_path}")
else:
    # Define the connection string
    conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=35.184.99.218;'
    r'DATABASE=coolsys;'
    r'UID=sqlserver;'
    r'PWD=Ybz8Vq+|>\H/<2py'
    )

    # Define the SQL query
    sql_query = """
    SELECT
        w.wrkordr_wrk_rqstd,
        w.wrkordr_wrk_prfrmd,
        w2.wrkordreqpmnt_wrk_rqstd,
        w2.wrkordreqpmnt_wrk_prfrmd,
        w3.wrkordrinvntry_dscrptn
    FROM
        coolsys.dbo.wrkordr w
    INNER JOIN coolsys.dbo.wrkordrinvntry w3 ON
        w.wrkordr_rn = w3.wrkordr_rn
    INNER JOIN coolsys.dbo.wrkordreqpmnt w2 ON
        w.wrkordr_rn = w2.wrkordr_rn
    WHERE
        w.wrkordr_wrk_rqstd LIKE '%ICE MACHINE%' OR
        w.wrkordr_wrk_prfrmd LIKE '%ICE MACHINE%' OR
        w2.wrkordreqpmnt_wrk_rqstd LIKE '%ICE MACHINE%' OR
        w2.wrkordreqpmnt_wrk_prfrmd LIKE '%ICE MACHINE%' OR
        w3.wrkordrinvntry_dscrptn LIKE '%ICE MACHINE%';
    """

    # Connect to the remote MSSQL database
    conn = pyodbc.connect(conn_str)
    print("Connection to the database was successful.")

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()

    # Execute the SQL query to retrieve all tables and columns
    cursor.execute(sql_query)
    rows = cursor.fetchall()

    # Get column names from the cursor description
    columns = [column[0] for column in cursor.description]

    if conn:
        # Close the connection
        conn.close()
        print("Connection closed.")

    # Convert the fetched data to a pandas DataFrame
    df = pd.DataFrame.from_records(rows, columns=columns)

    # Export the DataFrame to a CSV file
    df.to_csv(csv_file_path, index=False)
    print(f"DataFrame has been exported to {csv_file_path}")
print(df.shape)

DataFrame loaded from data/ice_makers.csv
(296547, 5)


In [5]:
print(df.shape)
df.head()

(296547, 5)


,wrkordr_wrk_rqstd,wrkordr_wrk_prfrmd,wrkordreqpmnt_wrk_rqstd,wrkordreqpmnt_wrk_prfrmd,wrkordrinvntry_dscrptn
0,PREVENTIVE MAINT -- 2012 JAN DEEP DIVE\rFILL...,NaN,PREVENTIVE MAINT -- 2012 JAN DEEP DIVE\rFILL...,ST713886-PERFORMED PER SCOPE-NO PROBLEMS NOTED,ICE MACHINE CLEANER NICKEL SAFE
1,PREVENTIVE MAINT -- 2012 JAN DEEP DIVE\rFILL...,NaN,PREVENTIVE MAINT -- 2012 JAN DEEP DIVE\rFILL...,ST713890 - PERFORMED PM PER SCOPE,ICE MACHINE CLEANER NICKEL SAFE
2,SERVICE CALL -- 2012 JAN DEEP DIVE\rFILL OUT...,NaN,SERVICE CALL -- 2012 JAN DEEP DIVE\rFILL OUT...,ST713888-PERFORMED PER SCOPE-NO PROBLEMS NOTED,ICE MACHINE CLEANER NICKEL SAFE
3,PREVENTIVE MAINT -- 2012 JAN DEEP DIVE\rFILL...,NaN,PREVENTIVE MAINT -- 2012 JAN DEEP DIVE\rFILL...,COMPLETED PER SCOPE - NO PROBLEMS NOTED - CMP ...,ICE MACHINE CLEANER NICKEL SAFE
4,ICE MACHINE NOT WORKING - MAKING LOUD NOISE WH...,I/M R/R COND FAN MOTOR,ICE MACHINE NOT WORKING - MAKING LOUD NOISE WH...,ST 212897 - REMOVED AND REPLACED COND FAN MOTO...,FAN MTR 240V 606/806/1006 3/4 MS X MS


In [6]:
# Define the strings to exclude from the DataFrame
exclude_strings = [
    'PREVENTIVE MAINT', 'PM SERVICE', 
    'Q1 MAINTENANCE', 'Q2 MAINTENANCE', 'Q3 MAINTENANCE','Q4 MAINTENANCE', 
    'NICKEL SAFE', 'COVID 19', 'SANITIZER'
]


# Create a boolean mask to identify rows where any column contains any of the search phrases
mask = df.apply(lambda row: any(phrase in str(x) for phrase in exclude_strings for x in row), axis=1)

# Filter the DataFrame to remove rows where any of the search phrases are present
df_filtered = df[~mask]

# Replace NaN values with empty strings
# df_filtered = df_filtered.fillna('')

print(df_filtered.shape)

(142141, 5)


In [7]:
# Combine the records from the specified columns into 'combined_column' without changing the original columns
df_filtered = df_filtered.copy()  # Create a copy to avoid SettingWithCopyWarning
df_filtered.loc[:, 'combined_column'] = df_filtered.apply(
    lambda row: ' '.join([str(row['wrkordr_wrk_prfrmd']) if pd.notna(row['wrkordr_wrk_prfrmd']) else '',
                          str(row['wrkordreqpmnt_wrk_prfrmd']) if pd.notna(row['wrkordreqpmnt_wrk_prfrmd']) else '',
                          str(row['wrkordrinvntry_dscrptn']) if pd.notna(row['wrkordrinvntry_dscrptn']) else '']),
    axis=1
)

In [8]:
# Replace non-alphanumeric characters with a space using .loc
df_filtered.loc[:, 'combined_column'] = df_filtered['combined_column'].str.replace(r'[^a-zA-Z0-9\s]', ' ', regex=True)


# Replace carriage return characters with a space using .loc
df_filtered.loc[:, 'combined_column'] = df_filtered['combined_column'].str.replace(r'\r', ' ', regex=True)


In [9]:
# Print the filtered DataFrame
df_filtered.head()

,wrkordr_wrk_rqstd,wrkordr_wrk_prfrmd,wrkordreqpmnt_wrk_rqstd,wrkordreqpmnt_wrk_prfrmd,wrkordrinvntry_dscrptn,combined_column
4,ICE MACHINE NOT WORKING - MAKING LOUD NOISE WH...,I/M R/R COND FAN MOTOR,ICE MACHINE NOT WORKING - MAKING LOUD NOISE WH...,ST 212897 - REMOVED AND REPLACED COND FAN MOTO...,FAN MTR 240V 606/806/1006 3/4 MS X MS,I M R R COND FAN MOTOR ST 212897 REMOVED AND...
5,ICE MACHINE NOT WORKING,NaN,ICE MACHINE NOT WORKING,INSTALLED 9 LBS R404 IN ICE MACHINE - UNIT IS ...,R404A FREON 24 LBS LT & MT HFC HP62,INSTALLED 9 LBS R404 IN ICE MACHINE UNIT IS...
6,ICE MACHINE NOT WORKING -- ERVICE CALL -- I/...,NaN,ICE MACHINE NOT WORKING -- ERVICE CALL -- I/...,EVAP HOT WAS BEEPING AND CUSTOMER RESET -- TRA...,R404A FREON 24 LBS LT & MT HFC HP62,EVAP HOT WAS BEEPING AND CUSTOMER RESET TR...
7,ICE MACHINE NOT WORKING -- REDISPATCH -- ICE...,NaN,ICE MACHINE NOT WORKING -- REDISPATCH -- ICE...,FOUND DIRECT SHORT BETWEEEN CONDENSER AND FAN ...,RELAY 2 POLE 240V,FOUND DIRECT SHORT BETWEEEN CONDENSER AND FAN...
8,ICE MACHINE NOT WORKING -- SERVICE CALL -- I...,NaN,ICE MACHINE NOT WORKING -- SERVICE CALL -- I...,FOUND THAT GAS VALVE LEAKING--NEED R/R IN AM D...,R404A FREON 24 LBS LT & MT HFC HP62,FOUND THAT GAS VALVE LEAKING NEED R R IN AM ...


In [10]:
# Select a random sample of 1000 records from the DataFrame
df_filtered = df_filtered.sample(n=1000, random_state=42)

In [11]:
# Import required library for progress bar
from tqdm.notebook import tqdm
from collections import defaultdict

def process_maintenance_data(df):
    """
    Process maintenance data with progress bar visualization
    """
    tasks_by_category = defaultdict(list)
    category_totals = defaultdict(int)
    task_frequencies = defaultdict(int)
    all_verb_object_pairs = []
    
    print("Starting maintenance data processing...")
    total_records = len(df['combined_column'].dropna())
    print(f"Total records to process: {total_records}")
    
    # Process records with progress bar
    with tqdm(total=total_records, desc='Processing Records', unit='record') as pbar:
        for text in df['combined_column'].dropna():
            # Process the text
            pairs = process_text(str(text), tokenizer, model, device)
            
            if pairs:
                for verb, obj in pairs:
                    category = categorize_task(verb, obj)
                    task = f"{verb}: {obj}"
                    
                    # Update frequencies and store pairs
                    task_frequencies[task] += 1
                    category_totals[category] += 1
                    all_verb_object_pairs.append((verb, obj))
                    
                    # Check if task already exists in category
                    task_exists = False
                    for existing_task in tasks_by_category[category]:
                        if existing_task[0] == task:
                            task_exists = True
                            break
                            
                    if not task_exists:
                        tasks_by_category[category].append(
                            (task, verb, task_frequencies[task])
                        )
            
            # Update progress bar
            pbar.update(1)
    
    print(f"\nProcessing complete!")
    print(f"Total task occurrences: {len(all_verb_object_pairs)}")
    print(f"Unique tasks: {len(task_frequencies)}")
    
    return tasks_by_category, category_totals, task_frequencies, all_verb_object_pairs

# Process the data with progress bar
tasks_by_category, category_totals, task_frequencies, all_pairs = process_maintenance_data(df_sample)

NameError: name 'df_sample' is not defined

In [11]:
def initialize_model():
    try:
        # Check if GPU/MPS is available and set the device
        if torch.backends.mps.is_available():
            device = torch.device("mps")
        else:
            device = torch.device("cpu")
        
        # Load the pre-trained BERT model for POS tagging
        model_name = 'vblagoje/bert-english-uncased-finetuned-pos'
        tokenizer = BertTokenizer.from_pretrained(model_name)
        model = BertForTokenClassification.from_pretrained(model_name).to(device)
        
        return tokenizer, model, device
    except Exception as e:
        print(f"Error initializing model: {str(e)}")
        return None, None, None

# Initialize the model
tokenizer, model, device = initialize_model()

In [12]:
def process_text(text, tokenizer, model, device):
    try:
        # Text preprocessing
        text = re.sub(r'\bas needed\b', '', text.lower())
        text = re.sub(r'\s*,\s*and\s+', ', ', text)  # Normalize conjunctions
        text = re.sub(r'\s+', ' ', text).strip()

        # Initialize the spell checker and lemmatizer
        spell = SpellChecker()
        lemmatizer = WordNetLemmatizer()

        # Process the text
        tokens = word_tokenize(text)
        tagged = pos_tag(tokens)
        
        # Extract verbs and their corresponding objects
        verb_object_pairs = []
        current_verb = None
        current_objects = []
        
        def is_potential_verb(word, tag):
            # Check if word ends with common past tense/participle endings
            return (tag.startswith('VB') or tag == 'VBD' or tag == 'VBN' or 
                   (tag == 'JJ' and (word.endswith('ed') or word.endswith('en'))))
        
        i = 0
        while i < len(tagged):
            word, tag = tagged[i]
            
            # Handle verbs (including past tense, participles, and adjectives that are actually verbs)
            if is_potential_verb(word, tag):
                # Save previous pair if exists
                if current_verb and current_objects:
                    verb_object_pairs.append((lemmatizer.lemmatize(current_verb, 'v'), ' '.join(current_objects)))
                current_verb = spell.correction(word)
                current_objects = []
            
            # Handle nouns, adjectives, and compound objects
            elif (tag.startswith('NN') or 
                  (tag.startswith('JJ') and not word.endswith('ed')) or 
                  tag == 'IN' or tag.startswith('VBG')):  # Include prepositions and gerunds
                if current_verb:
                    temp_objects = [word]
                    # Look ahead for compound objects and their modifiers
                    j = i + 1
                    while j < len(tagged) and (
                        tagged[j][1].startswith('NN') or 
                        (tagged[j][1].startswith('JJ') and not tagged[j][0].endswith('ed')) or 
                        tagged[j][1] == 'IN' or  # Include prepositions
                        tagged[j][1].startswith('VBG')  # Include gerunds
                    ):
                        temp_objects.append(tagged[j][0])
                        j += 1
                    
                    # Only add if we have a meaningful object phrase
                    if any(t[1].startswith('NN') for t in tagged[i:j]):
                        current_objects.extend(temp_objects)
                    i = j - 1  # Update index to skip processed compound words
            
            # Handle commas as phrase separators
            elif word == ',':
                if current_verb and current_objects:
                    verb_object_pairs.append((lemmatizer.lemmatize(current_verb, 'v'), ' '.join(current_objects)))
                    current_objects = []
                    current_verb = None
            
            i += 1

        # Add the last pair if exists
        if current_verb and current_objects:
            verb_object_pairs.append((lemmatizer.lemmatize(current_verb, 'v'), ' '.join(current_objects)))

        return verb_object_pairs

    except Exception as e:
        print(f"Error processing text: {str(e)}")
        return []

In [13]:
def categorize_task(verb, obj):
    """Categorize a task based on its verb into one of four categories: Inspection, Cleaning, Maintenance, or Other."""
    inspection_verbs = {'inspect', 'check', 'examine', 'monitor', 'observe', 'verify', 'test', 'diagnose', 'assess'}
    cleaning_verbs = {'clean', 'brush', 'wash', 'wipe', 'descale', 'sanitize', 'flush', 'clear'}
    maintenance_verbs = {'lubricate', 'adjust', 'replace', 'repair', 'calibrate', 'install', 'fix', 'tighten', 'service'}
    
    verb = verb.lower()
    if verb in inspection_verbs:
        return "Inspection"
    elif verb in cleaning_verbs:
        return "Cleaning"
    elif verb in maintenance_verbs:
        return "Maintenance"
    else:
        return "Other"

In [ ]:
# Process all records in df_filtered['combined_column']
from collections import Counter

print("Starting to process maintenance records...")

# Initialize counters for tasks and categories
all_verb_object_pairs = []  # Keep as list to track frequencies
task_frequencies = Counter()

# Process each record
total_records = len(df_filtered['combined_column'].dropna())
print(f"Total records to process: {total_records}")

for idx, text in enumerate(df_filtered['combined_column'].dropna(), 1):
    if idx % 100 == 0:  # Print progress every 100 records
        print(f"Processing record {idx}/{total_records} ({(idx/total_records*100):.1f}%)")
    
    pairs = process_text(text, tokenizer, model, device)
    if pairs:  # Only add if we got valid pairs
        for pair in pairs:
            task_frequencies[tuple(pair)] += 1
            all_verb_object_pairs.append(tuple(pair))

print(f"\nTotal task occurrences: {len(all_verb_object_pairs)}")
print(f"Unique tasks: {len(task_frequencies)}")

# Group tasks by category with frequencies
tasks_by_category = {}
category_totals = {}

for (verb, obj), freq in task_frequencies.most_common():
    category = categorize_task(verb, obj)
    if category not in tasks_by_category:
        tasks_by_category[category] = []
        category_totals[category] = 0
    tasks_by_category[category].append((verb, obj, freq))
    category_totals[category] += freq

# Print tasks by category with frequencies
print("\nTask Categories Analysis")
print("=" * 50)

total_tasks = sum(category_totals.values())

for category in ["Other", "Maintenance", "Inspection", "Cleaning"]:
    if category in tasks_by_category:
        print(f"\n{category} Tasks ({category_totals[category]} occurrences, {(category_totals[category]/total_tasks*100):.1f}%)")
        print("-" * 40)
        
        # Sort tasks by frequency within category
        sorted_tasks = sorted(tasks_by_category[category], key=lambda x: (-x[2], x[0]))
        
        for verb, obj, freq in sorted_tasks:
            percentage = (freq / category_totals[category]) * 100
            print(f"• {verb.capitalize()}: {obj} ({freq} times, {percentage:.1f}% of {category})")

# Print overall statistics
print("\nTask Distribution Statistics")
print("=" * 50)
for category, total in category_totals.items():
    percentage = (total / total_tasks) * 100
    unique_tasks = len(tasks_by_category[category])
    print(f"{category}: {total} occurrences ({percentage:.1f}%), {unique_tasks} unique tasks")

# Create a summary DataFrame
summary_data = {
    'Category': [],
    'Total_Occurrences': [],
    'Unique_Tasks': [],
    'Percentage': []
}

for category in tasks_by_category:
    summary_data['Category'].append(category)
    summary_data['Total_Occurrences'].append(category_totals[category])
    summary_data['Unique_Tasks'].append(len(tasks_by_category[category]))
    summary_data['Percentage'].append((category_totals[category] / total_tasks) * 100)

summary_df = pd.DataFrame(summary_data)
print("\nSummary DataFrame:")
print("=" * 50)
print(summary_df.to_string(index=False, float_format=lambda x: '{:.1f}'.format(x)))

Starting to process maintenance records...
Total records to process: 10000
Processing record 100/10000 (1.0%)
Processing record 200/10000 (2.0%)
Processing record 300/10000 (3.0%)
Processing record 400/10000 (4.0%)
Processing record 500/10000 (5.0%)
Processing record 600/10000 (6.0%)
Processing record 700/10000 (7.0%)
Processing record 800/10000 (8.0%)
Processing record 900/10000 (9.0%)
Processing record 1000/10000 (10.0%)
Processing record 1100/10000 (11.0%)
Processing record 1200/10000 (12.0%)
Processing record 1300/10000 (13.0%)
